<a href="https://colab.research.google.com/github/kimsojeong1225/crawling/blob/master/%EC%A3%BC%EA%B0%80_%EC%A1%B0%ED%9A%8C_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd

### krx 사이트에 등록되어 있는 상장사 정보(html) 다운로드 받아 dataframe 객체 생성
* krx 상장사 정보 url : http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13

In [0]:
code_df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13', header=0)[0]

In [3]:
code_df.head(5)

,회사명,종목코드,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역
0,DSR,155660,1차 비철금속 제조업,합섬섬유로프,2013-05-15,12월,홍석빈,http://www.dsr.com,부산광역시
1,GS글로벌,1250,상품 종합 도매업,"수출입업(시멘트,철강금속,전기전자,섬유,기계화학),상품중개,광업,채석업/하수처리 서...",1976-06-26,12월,김태형,http://www.gsgcorp.com,서울특별시
2,HSD엔진,82740,일반 목적용 기계 제조업,"대형선박용엔진,내연발전엔진",2011-01-04,12월,고영열,http://www.doosanengine.com,경상남도
3,LG이노텍,11070,전자부품 제조업,기타 전자부품 제조업,2008-07-24,12월,정철동,http://www.lginnotek.co.kr,서울특별시
4,OCI,10060,기초 화학물질 제조업,"타르제품,카본블랙,무수프탈산,농약원제,석탄화학제품,정밀화학제품,플라스틱창호재 제조,판매",1985-07-09,12월,"백우석, 이우현, 김택중(3인, 각자 대표이사)",http://www.oci.co.kr,서울특별시


In [4]:
code_df['종목코드'].head(5)

0    155660
1      1250
2     82740
3     11070
4     10060
Name: 종목코드, dtype: int64

### Q1. 상장사 정보 중 종목코드를 왼쪽이 '0' 으로 채워진 6자리 값으로 변환하시오

In [0]:
code_df['종목코드']=code_df['종목코드'].map('{:06d}'.format)

In [6]:
code_df.head(5)

,회사명,종목코드,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역
0,DSR,155660,1차 비철금속 제조업,합섬섬유로프,2013-05-15,12월,홍석빈,http://www.dsr.com,부산광역시
1,GS글로벌,001250,상품 종합 도매업,"수출입업(시멘트,철강금속,전기전자,섬유,기계화학),상품중개,광업,채석업/하수처리 서...",1976-06-26,12월,김태형,http://www.gsgcorp.com,서울특별시
2,HSD엔진,082740,일반 목적용 기계 제조업,"대형선박용엔진,내연발전엔진",2011-01-04,12월,고영열,http://www.doosanengine.com,경상남도
3,LG이노텍,011070,전자부품 제조업,기타 전자부품 제조업,2008-07-24,12월,정철동,http://www.lginnotek.co.kr,서울특별시
4,OCI,010060,기초 화학물질 제조업,"타르제품,카본블랙,무수프탈산,농약원제,석탄화학제품,정밀화학제품,플라스틱창호재 제조,판매",1985-07-09,12월,"백우석, 이우현, 김택중(3인, 각자 대표이사)",http://www.oci.co.kr,서울특별시


### Q2. code_df 객체에서 '회사명', '종목코드' 컬러만 선택하여 code_df 객체에 재 할당 하시오.

In [7]:
code_df=code_df[['회사명','종목코드']]
code_df

,회사명,종목코드
0,DSR,155660
1,GS글로벌,001250
2,HSD엔진,082740
3,LG이노텍,011070
4,OCI,010060
...,...,...
2349,지앤이헬스케어,299480
2350,코셋,189350
2351,틸론,217880
2352,판도라티비,202960


### Q3. code_df 객체에서 '회사명' , '종목코드' 컬럼의 이름을 각 각 'name', 'code' 로 변경 하시오

In [8]:
code_df.rename(columns={'회사명':'name','종목코드':'code'},inplace=True)
code_df

,name,code
0,DSR,155660
1,GS글로벌,001250
2,HSD엔진,082740
3,LG이노텍,011070
4,OCI,010060
...,...,...
2349,지앤이헬스케어,299480
2350,코셋,189350
2351,틸론,217880
2352,판도라티비,202960


### Q4. get_url 함수를 정의하시오
* 입럭받은 '회사명'(item_name) 정보와 code_df 객체를 이용하여 '상장 코드' 를 이용하여 naver 주식 시세 정보를 검색하는 url을 작성하시오
* naver 주식 시세 정보 검색 url : https://finance.naver.com/item/sise_day.nhn?code=`상장 코드`
* 작성한 함수를 이용하여 삼성전자 주식 조회를 위한 naver 주식 시세 정보 검색 url 정보를 출력하시오

In [0]:
def get_url(item_name, code_df):
  code=code_df['code'][code_df['name']==item_name].values[0]
  url= 'https://finance.naver.com/item/sise_day.nhn?code={}'.format(code)

  return url

In [0]:
# 삼성전자 주식 조회 를 위한 naver 주식 시세정보 검색 url 정보 출력
url=get_url('삼성전자',code_df)
item_name='삼성전자'

### url을 이용한 주식시세 정보 호출 및 html 파싱을 위한 bs4 라이브러리 임포트

In [0]:
import requests
from bs4 import BeautifulSoup

### Q5. Q4에서 생성한 naver 주식 시세정보 조회 url을 이용하여 200일 간의 삼성전자의 주식 시세 정보를 리스트 객체에 저장하시오
* list 의 shape = (200(일수), 7(날짜, 종가, 차액, 시가, 상한, 하한, 거래량))

In [0]:
import re
df = pd.DataFrame()

priceList = []

for page in range(1, 21):
  page_url='{url}&page={page}'.format(url=url,page=page)
  
  response=requests.get(page_url)
  soup=BeautifulSoup(response.content,'lxml')

  trList=soup.body.find_all('tr',{'onmouseover':'mouseOver(this)'})
  for i in range(0,10):
    a=trList[i].find_all(name='span')
    List=[]
    for j in range(0,len(a)):
      ret = re.sub('[-=.#/?:$}\n\t]', '', a[j].text)
      List.append(ret)
    priceList.append(List)



### Q6. priceList 정보를 이용하여 df 이름을 dataframe 객체 생성
* 컬럼의 이름은 'date', 'close', 'diff', 'open', 'high', 'low', 'volume' 롤 지정

In [0]:
col=[ 'date', 'close', 'diff', 'open', 'high', 'low', 'volume']
df=pd.DataFrame(priceList,columns=col)

### Q7. df 객체에서 Nan 제거

In [15]:
df.dropna

<bound method DataFrame.dropna of          date   close   diff    open    high     low      volume
0    20200114  60,000      0  60,400  61,000  59,900  16,084,418
1    20200113  60,000    500  59,600  60,000  59,100  11,359,139
2    20200110  59,500    900  58,800  59,700  58,300  16,000,170
3    20200109  58,600  1,800  58,400  58,600  57,400  24,102,579
4    20200108  56,800  1,000  56,200  57,400  55,900  23,501,171
..        ...     ...    ...     ...     ...     ...         ...
195  20190401  45,050    400  45,200  45,450  44,850   7,362,129
196  20190329  44,650    200  44,500  44,900  44,200  11,491,713
197  20190328  44,850    500  44,950  45,200  44,300   6,821,306
198  20190327  45,350    100  44,750  45,600  44,250   9,568,081
199  20190326  45,250    250  45,500  45,700  44,900   9,729,811

[200 rows x 7 columns]>

### df 객체에 각 원소들의 값이 object로 되어 있기 때문에 분석을 위해 float type으로 

In [16]:
# df 객체 내 컬럼들의 데이터 타입 조회(object))
df.dtypes

date      object
close     object
diff      object
open      object
high      object
low       object
volume    object
dtype: object

In [0]:
# 'close', 'diff', 'open', 'high', 'low', 'volume' 의 값들이
# 천 단위 표현을 위해 ','(5,000) 사용 하고 있음
# 해당 포멧을 숫자로 변환하기 위헤 locale 라이브러리 사용
import locale
from locale import atof

locale.setlocale(locale.LC_NUMERIC, '')

# 날짜의 경우 datetime 타입으로 변환
df['date'] = pd.to_datetime(df['date'])

df[['close', 'diff', 'open', 'high', 'low', 'volume']] = df[['close', 'diff', 'open', 'high', 'low', 'volume']].applymap(atof)
df = df.sort_values(by=['date'], ascending=True)

In [18]:
# df 객체 들의 변환된 데이터 타입 정보 조회(datetime, float64)
df.dtypes

date      datetime64[ns]
close            float64
diff             float64
open             float64
high             float64
low              float64
volume           float64
dtype: object

### 삼성 전자 종가 기준 일별 주가 시계열 차트 조회

In [0]:
# 시계열 차트 생성을 위한 라이브러리 임포트
import plotly.graph_objects as go

In [0]:
# 차트 생성을 위한 데이터 생성
trace = go.Scatter(x=df['date'],
                   y=df['close'],
                   name=item_name)

data = [trace]

In [0]:
# 시계열 차트의 레이웃 정보 생성
layout = dict(
    title='{}의 종가(close) Time Series'.format(item_name),
    # x축 데이터 타입 지정(date)
    xaxis = dict(
        rangeslider=dict(),
        type='date'
        ),
)

In [26]:
# 차트 객체 생성 및 조회
fig = go.Figure(data=data, layout=layout)
fig.show()